Create preprocessed tables for visualization

Table reviews_x_product

In [3]:
%sql
CREATE TABLE OR REPLACE reviews_x_product
    COMMENT 'View for reviews with product information'
    AS select A.*,
    B.alsobuy_0,
    B.alsoview_1,
    B.asin_2,
    B.brand_3,
    B.category_4,
    B.date_5,
    B.description_6,
    B.feature_7,
    B.fit_8,
    B.image_9,
    B.maincat_10,
    B.price_11,
    B.rank_12,
    B.similaritem_13,
    B.tech1_14,
    B.tech2_15,
    B.title_16,
    B.ItemWeight_17,
    B.PackageDimensions_18,
    B.ProductDimensions_19,
    B.DatefirstlistedonAmazon_20,
    B.UNSPSCCode_21
    FROM reviews A
    inner join products_metadata B
    ON A.asin=B.asin_2;

Table summary_score_reviews

In [5]:
%sql
CREATE OR REPLACE TABLE summary_score_reviews AS
select to_date(from_unixtime(unixReviewTime, 'MM/yyyy'),"MM/yyyy") as date, maincat_10, verified, ROUND(avg(overall),2) as Average, min(overall) as Min_rate, max(overall) as Max_rate, ROUND(sum(overall)/(5*count(overall)),2) as efficiency_rate, count(overall) as Qty, ROUND(stddev(overall),2) as sDeviation
FROM reviews_x_product
group by date,maincat_10, verified
order by date;

In [6]:
%sql 
DROP VIEW reviews_x_product

In [7]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import pandas as pd

# Azure Blob Storage credentials
storage_connection_string = "DefaultEndpointsProtocol=https;AccountName=test1fast;AccountKey=QnSkjChqVUQWCLs9t+yDSK4w02oQVBjWtP9dOOBhpw1O002GrWnk8LHfsU8Ys16QjNKmjnDw2RbM+AStEQNjww==;EndpointSuffix=core.windows.net"
container_name = "visualization-tables"

# Connect to Azure Blob Storage
blob_service_client = BlobServiceClient.from_connection_string(storage_connection_string)
container_client = blob_service_client.get_container_client(container_name)
blob_client = container_client.get_blob_client(csv_filename)

In [8]:
df = pd.read_csv(csv_filename)
df

,AAPL_x,AAPL_y
0,2014-01-02,77.445395
1,2014-01-03,77.045575
2,2014-01-06,74.896972
3,2014-01-07,75.856461
4,2014-01-08,75.091947
...,...,...
235,2014-12-08,113.653345
236,2014-12-09,109.755497
237,2014-12-10,113.960331
238,2014-12-11,111.817477


In [9]:
#Save csv on Azure Blob Storage
with open("stockPrice.csv", mode="rb") as data:
    blob_client = container_client.upload_blob(name="stockPrice.csv", data=data, overwrite=True)